# Предобработка

In [32]:
import pandas as pd

In [18]:
df_0 = pd.read_csv("pm fmkn upd.csv")

In [20]:
df = df_0.drop(columns = ['Unnamed: 0'],axis = 1)

In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pymorphy2
import string
import re

tokenizer = ToktokTokenizer()
stop = set(stopwords.words('russian'))

def preproc_df(text):
    for i in range(1, len(text)):
        morph = pymorphy2.MorphAnalyzer()
        text[i] = re.sub(r'\n', '', text[i])
        text[i] = re.sub(r'\d+', '', text[i])
        text[i] = text[i].translate(str.maketrans('','', string.punctuation))
        text[i] = ' '.join([morph.parse(word)[0].normal_form for word in text[i].split()])
        tokens = tokenizer.tokenize(text[i])
        tokens = [token.strip() for token in tokens]
        cleaned_tokens = [token for token in tokens if token.lower() not in stop]
        text[i] = ' '.join(cleaned_tokens)           
    return text

In [22]:
df = df.apply(preproc)

In [23]:
df = df.fillna('')

In [25]:
df.to_csv("pm fmkn upd.csv", index=False)

In [1]:
import pandas as pd
df = pd.read_csv("pm fmkn upd.csv")
df = df.fillna('')

In [2]:
df_1 = df.to_numpy().reshape(4824*31)

In [3]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

vector_size = 30
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(df_1)]
model = Doc2Vec(documents, vector_size=vector_size, window=2, min_count=1, workers=4, dm=0)
model.build_vocab(documents)

In [4]:
for epoch in range(10):
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.epochs)

In [5]:
model.save("pm_fmkn_trained upd dm0")

# Проверка векторизации

In [6]:
from gensim.models.doc2vec import Doc2Vec
model = Doc2Vec.load("pm_fmkn_trained upd dm0")

In [7]:
import pandas as pd
mm = pd.read_csv('math-mech upd.csv')

In [8]:
mm.drop(index=0)

,Титульная страница,Раздел 1.,1.1.,1.2.,1.3.,1.4.,Раздел 2.,2.1.,2.1.1,2.2.,...,3.3.1,3.3.2,3.3.3,3.3.4,3.3.5,3.4.,3.4.1,3.4.2,3.4.3,Раздел 4.
1,санкт-петербургский государственный университе...,NaN,основный цель обучение иностранный обучаться р...,"обучающиеся, приступать к изучение курс русски...",по завершение курс обучение у учиться должный ...,основной форма обучение русский язык являться ...,NaN,предусмотреть учебный занятие с использование ...,NaN,каждый учебный год иметь семестровый и модульн...,...,стандартно оборудовать аудитория для практичес...,компьютерный класс с выход в internet.,не предусмотреть,не предусмотреть,не предусмотреть,NaN,1. барсуков-сергеев о.м. знакомый глаголы: пос...,"акишин т.е., алексеев н.н. пособие по обучение...",филологический портал: philology.ru справочно-...,"волков л.б., канд. филол. наук, доцент, доцент..."
2,санкт-петербургский государственный университе...,NaN,NaN,NaN,NaN,NaN,NaN,предусмотреть учебный занятие с использование ...,NaN,l-й блок «спортивный» l-й модуль «самосовершен...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,санкт-петербургский государственный университе...,NaN,цель и задача обучение английский язык формули...,уровень подготовленность обучаться определятьс...,в результат освоение программа обучаться должн...,"в связь с тем, что данный курс являться коммун...",NaN,предусмотреть учебный занятие с использование ...,NaN,NaN,...,в аудитория обязательно наличие доски. при про...,всё рабочий станция должный быть оснастить лиц...,специализировать лингафонный и мультимедийный ...,определяться в соответствие с состав закупка в...,расходный материал обеспечиваться в соответств...,NaN,определяться при составление профессионально-о...,общий курс (по выбор преподавателя): 1. amanda...,наиболее полный перечень информационный источн...,"кузнецов людмила борисовна, к.ф.н., доцент, до..."
4,санкт-петербургский государственный университе...,NaN,цель и задача обучение английский язык формули...,наличие компетенция в предел уровень в2.,в результат освоение программа обучаться должн...,"в связь с тем, что данный курс являться коммун...",NaN,NaN,NaN,для обучаться высокий уровень сформированность...,...,"в аудитория обязательно наличие доски, а также...",всё рабочий станция должный быть оснастить лиц...,специализировать лингафонный и мультимедийный ...,определяться в соответствие с требование текущ...,расходный материал обеспечиваться в соответств...,NaN,определяться при составление профессионально-о...,"1. puhta, h., thaine, g., doff, a., stranks, j...",наиболее полный перечень информационный источн...,"ибрагимов динар хамидовна, кафедра иностранный..."
5,санкт-петербургский государственный университе...,NaN,дисциплина «математический статистика» являтьс...,программа курс предназначить обучаться на 4 ку...,обучаться должны: • знать содержание дисциплин...,аудиторный учебный работа: теоретический занят...,NaN,NaN,NaN,тема для изучение и обсуждение 1) классификаци...,...,стандартно оборудовать лекционный аудитория. д...,стандартный аудиторный оборудование и по.,не требуется.,не требуется.,мел для доски.,NaN,1. боровков а.а. математический статистика. - ...,1. крамер г. математический метод статистики. ...,не требуется.,"никитин яков юрьевич, д.ф-м.н, проф. кафедра т..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,санкт-петербургский государственный университе...,NaN,дисциплина «русский язык как иностранный» пред...,уровень подготовленность обучаться в область р...,в результат освоение программа по траектория 1...,основной форма обучение русский язык являться ...,NaN,NaN,NaN,каждый учебный год иметь семестровый и модульн...,...,a) для проведение занятие по русский язык целе...,"наличие аудиовизуальный техники, включая мульт...",мультимедийный учебный комплекс с сильный прог...,1) компьютерный программы: microsoftoffice (в ...,возможность ксерокопирование учебный материалов.,"барыкин а.н., добровольский в.в.изучать глагол...",NaN,аникин м.н. си

In [9]:
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
import pymorphy2
import my_parser
import pandas as pd
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re


obr_1 = 'рпд-мм/001292_Теория представлений.docx'
# obr_2 = 
# obr_3 =

cols = [column for column in mm]

def preproc_rpd(text):
    tokenizer = ToktokTokenizer()
    stopword_list = nltk.corpus.stopwords.words('russian')
    stop = set(stopwords.words('russian'))
    text = text['text']
    morph = pymorphy2.MorphAnalyzer()
    text = ' '.join([morph.parse(word)[0].normal_form for word in text.split()])
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    cleaned_tokens = [token for token in tokens if token.lower() not in stopword_list]
    cleaned_text = ' '.join(cleaned_tokens)           
    return cleaned_text

def process(obr):
    rpd_content = my_parser.feed_content(obr)
    rpd = pd.DataFrame(rpd_content)
    rpd = rpd.drop(index='title')
    rpd = rpd.apply(preproc_rpd)
    return rpd
    
def check(obr):
    obr = process(obr)
    closest = dict()
    mm = pd.read_csv('math-mech upd.csv')
    mm = mm.fillna('')
    
    for c in cols:
        closest[c] = list()
        min = 1000
        for i in range(1, len(mm)):
            v_1 = model.infer_vector(obr[c].split()).reshape(1, -1)
            v_2 = model.infer_vector(mm.loc[i][c].split()).reshape(1, -1)
            cos_dist = 1 - cosine_similarity(v_1, v_2)
            if cos_dist < min:
                min = cos_dist
                closest[c].append([i, cos_dist[0]])
    return closest


check(obr_1)

{'Титульная страница': [[1, array([0.21975327], dtype=float32)],
  [3, array([0.19774354], dtype=float32)],
  [5, array([0.19666427], dtype=float32)],
  [12, array([0.18067086], dtype=float32)],
  [19, array([0.13869071], dtype=float32)],
  [20, array([0.12754011], dtype=float32)],
  [681, array([0.12331271], dtype=float32)],
  [728, array([0.11972678], dtype=float32)],
  [904, array([0.11002815], dtype=float32)]],
 'Раздел 1.': [[1, array([1.1886629], dtype=float32)],
  [2, array([0.91054434], dtype=float32)],
  [3, array([0.74929094], dtype=float32)],
  [14, array([0.6646794], dtype=float32)],
  [29, array([0.6378196], dtype=float32)],
  [35, array([0.53615594], dtype=float32)]],
 '1.1.': [[1, array([0.5823618], dtype=float32)],
  [5, array([0.52625465], dtype=float32)],
  [7, array([0.37714404], dtype=float32)],
  [10, array([0.31738037], dtype=float32)],
  [14, array([0.3142041], dtype=float32)],
  [20, array([0.05200505], dtype=float32)]],
 '1.2.': [[1, array([0.59170043], dtype=f

In [10]:
obr_1 = 'рпд-мм/001292_Теория представлений.docx'
obr_2 = 'рпд-мм/002051 Практикум на ЭВМ.docx'
r_1 = process(obr_1)
r_2 = process(obr_1)
r_3 = process(obr_2)
for c in cols:
    if r_1[c] != '' and r_2[c] != '' and r_3[c] != '':
        v_1 = model.infer_vector(r_1[c].split()).reshape(1, -1)
        v_2 = model.infer_vector(r_2[c].split()).reshape(1, -1)
        v_3 = model.infer_vector(r_3[c].split()).reshape(1, -1)
#         cos_dist = cosine_similarity(v_1, v_2)
        print(c)
        print('Itself', cosine_similarity(v_1, v_1))
        print('Itself but diff doc', cosine_similarity(v_1, v_2))
        print('Different', cosine_similarity(v_1, v_3))
        print('Different 2', cosine_similarity(v_2, v_3))
        

Титульная страница
Itself [[0.99999994]]
Itself but diff doc [[0.98995924]]
Different [[0.88349324]]
Different 2 [[0.8744291]]
1.1.
Itself [[1.]]
Itself but diff doc [[0.98868215]]
Different [[0.46548605]]
Different 2 [[0.45616126]]
1.2.
Itself [[0.99999994]]
Itself but diff doc [[0.96426815]]
Different [[0.64956]]
Different 2 [[0.66455734]]
1.3.
Itself [[0.9999999]]
Itself but diff doc [[0.99581635]]
Different [[0.6053561]]
Different 2 [[0.6216016]]
1.4.
Itself [[0.99999994]]
Itself but diff doc [[0.96781486]]
Different [[0.60988957]]
Different 2 [[0.57300174]]
2.2.
Itself [[1.]]
Itself but diff doc [[0.99540603]]
Different [[0.46655995]]
Different 2 [[0.4685189]]
3.1.1
Itself [[0.99999994]]
Itself but diff doc [[0.9429818]]
Different [[0.56331575]]
Different 2 [[0.56165874]]
3.1.2
Itself [[0.9999999]]
Itself but diff doc [[0.97428966]]
Different [[0.5820201]]
Different 2 [[0.6013048]]
3.1.3
Itself [[0.9999999]]
Itself but diff doc [[0.9920636]]
Different [[0.6158727]]
Different 2 [[0